**Apply BBO**

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import datasets
from sklearn.metrics import accuracy_score
from random import choices , randint, randrange , random,uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier



# Define the BBO algorithm parameters
population_size = [5]          #automate
generation_size= [10]
m = 0.01            #User Define mutation Parameter

#this function will generate a list of 0 and 1,of lenght equal to the total number of habitat in dataset
#basically it will create our habitat..
# num_features = x_df.shape[1]    #number of features
def generate_habitat(x_df):
    num_features = x_df.shape[1]    #number of features
    return [randint(0, 1) for _ in range(num_features)]

#this population will create our population
#it will call generate_habitat() function number of times = population_size and and append all chromosome into list return a population

def init_population(p,x_df):
  population = []
  for i in range(p):
    habitat=generate_habitat(x_df)
    select_feture_column=select_feature(habitat)
    population.append(select_feture_column)
  return population

# convert actual habitat into seleccted features dataframe
# this function will iterate over our habitat and the indices it will find 1, it will add the feature of that indices to our
# new DataFrame
def select_feature(habitat):
   ct=0
   for i in habitat:        #if habitat generate[0,0,0,0,......] No feature selected thorw error
     if i==0:
      ct+=1
   if ct==len(habitat):          #if count is equal to length of habitat flip 0th bit
     habitat[0]=1
   feature_column=[]
   for i in range(len(habitat)):
      if(habitat[i]==1):
         feature_column.append(i)
  #  print(feature_column)
   return feature_column

def select_feature_DataFrame(habitat,x_df):
  # print(x_df.shape[1])
  new_df=pd.DataFrame()
  for i in habitat:
      new_df = pd.concat([new_df, x_df[x_df.columns[i]]], axis=1)
  return new_df

def fitness_function(habitat,x_df,y_df):
  select_feature_df=pd.DataFrame()
  select_feature_df=select_feature_DataFrame(habitat,x_df)      #returns selected features DataFrame

  xtrain , xtest, ytrain, ytest = train_test_split(select_feature_df, y_df,test_size=0.30,random_state=30)            #split selected feature dataframe traing and testing
  clf= RandomForestClassifier(max_depth=None,min_samples_leaf=1,min_samples_split=2,n_estimators=150)
  # score = np.mean(cross_val_score(clf,x_df,y_df.values.ravel(),cv=15))
  clf.fit(xtrain,ytrain.values.ravel())      #train classifier
  y_predict=clf.predict(xtest)
  score=accuracy_score(ytest,y_predict)    #calculate accuracy score between test target and predicted target
  return score

def immigration(sorted_indices,population_len):
  immigr=np.zeros([len(sorted_indices),1],dtype="float")
  for i in range(len(sorted_indices)):
    k=i+1
    lemda_k=(1-(k/population_len))
    immigr[sorted_indices[i],0]=lemda_k
  return immigr


def emmigration(sorted_indices,population_len):
  emmigr=np.zeros([population_len,1],dtype="float")
  for i in range(len(sorted_indices)):
    k=i+1
    lemda_u=(k/population_len)
    emmigr[sorted_indices[i],0]=lemda_u
    # print(emmigr[sorted_indices[i],0])
  return emmigr

#this function used to calculate mutation rate of each habitate in population by formula m(s) = m * ((1 - p(s))/p(max))
"""m(s): the mutation rate for the solution s
m: the mutation rate parameter, which is a constant value
p(s): the fitness probability of solution s
p(max): the maximum fitness probability in the population

p(s) = f(s) / sum(f(i))
p(s) probability of fitness score each fitness score divide by total fitness score """    #comment
def mutation_rate_compute(update_pop,population_len,x_df,y_df):
   fitness_score=[]
   for i in range(len(update_pop)):
        score=fitness_function(update_pop[i],x_df,y_df)
        fitness_score.append(score*100)                #calculate fitness score
   sorted_indices = np.argsort(fitness_score)[::-1]  #sort in descending order
   muted_rate_arr=np.zeros([population_len,1],dtype="float")           #create an array that store mutation rate with corresponding Habitate index
  #  total_habitate_fitness=sum(fitness_score)           #sum up all fitnesss score for calculating probability
   Ps=[]                                #empty list for calculating each habitate fitness probability
  #  for score in fitness_score:
  #         Ps.append(score/total_habitate_fitness)            #calculate fitness probability and append into a list
   for j in range(len(sorted_indices)):
          k=j+1
          Ps.append(k/len(update_pop))
   Pmax=max(Ps)                                                #also find maximum probability
   for i in range(len(update_pop)):
     mutation_rate=m*((1-Ps[i])/Pmax)                            #apply formula for evaluate mutation rate for each habitate
     muted_rate_arr[i,0]=mutation_rate                 #append it to an array

   return muted_rate_arr                                       #return mutation rate array




# n_features = x_df.shape[1]     # number of features
#perfrom mutation
def new_mutation(muted_rate_arr,pop,x_df,y_df):
  for i in range(len(pop)):
    n_features = x_df.shape[1]
    if uniform(0,1)<muted_rate_arr[i,0]:
      prev_score=fitness_function(pop[i],x_df,y_df)               #calculate score of habitat without mutation
      prev_habitat=pop[i]                               #store previous habitat without mutation
      indx=randint(0,len(pop[i])-1)        #generate an index in betwwen 0 to length of habitate
      num=randint(0,n_features-1)           #genetrate a number 0 to number of features
      if num not in pop[i]:                  #check if number is not present in habitate
        pop[i][indx]=num                     #then replace Habitate SIV with number
      update_score=fitness_function(pop[i],x_df,y_df)   #calculate habitat score after mutation
      if(update_score<prev_score):            #if after mutation habitat score increases then change the current habitat
         pop[i]=prev_habitat
  return pop                                  #return population


# n_features = x_df.shape[1]     # number of features
def new_migration(pop,immigr,emmigr,x_df):
  n_features = x_df.shape[1]     # number of features
  for i in range(len(pop)):
    if uniform(0,1)<immigr[i,0]:
      for j in range(len(pop)):
        if uniform(0,1)<emmigr[i,0]:
          # print("before migration pop[j]=",pop[i])
          if(len(pop[i])<=len(pop[j])):
            H_E_indx=randint(0,len(pop[i])-1)
            H_I_indx=randint(0,len(pop[i])-1)
          else:
             H_E_indx=randint(0,len(pop[j])-1)
             H_I_indx=randint(0,len(pop[j])-1)
          p1=pop[j][H_E_indx]        #emigration element extract
          if p1 not in pop[i]:           #if emigrated SIV is not present in Immigrated Habitate
             pop[i][H_I_indx]=p1         #replace SIV in immegrated habitate
          else:
            pop[i]=pop[i]             #else not replace immigrated habitate

  return pop                # return population


def runBBO(population_len,generation,x_df,y_df):
    pop=init_population(population_len,x_df)
    fitness_score=[]
    for i in range(len(pop)):
      score=fitness_function(pop[i],x_df,y_df)
      fitness_score.append(score)                      #calculate fitness score for initial population
      sorted_indices = np.argsort(fitness_score)[::-1]  #sort in descending order
    t=0
    list_ans = []
    list_ans.append([pop[sorted_indices[0]],fitness_function(pop[sorted_indices[0]],x_df,y_df)])  #append initial population bestest
    while t<generation:
        # print("generation=",t)
        immi_arr = immigration(sorted_indices,population_len)
        emmi_arr = emmigration(sorted_indices,population_len)
        update_pop=new_migration(pop, immi_arr, emmi_arr,x_df)        #perform migration  and return updated population
        mutation_rate=mutation_rate_compute(update_pop,population_len,x_df,y_df)       #compute mutation rate of updated population
        muteted_pop=new_mutation(mutation_rate,update_pop,x_df,y_df)         #perform mutaion and return mutated population
        fitness_score=[]
        for i in range(len(update_pop)):
           score=fitness_function(update_pop[i],x_df,y_df)
           fitness_score.append(score)                        #again calculate fitness score
        sorted_indices = np.argsort(fitness_score)[::-1]  #sort in descending order
        list_ans.append([update_pop[sorted_indices[0]],fitness_function(update_pop[sorted_indices[0]],x_df,y_df)])   #store best one
        pop=update_pop       #initialize pop to update population
        t=t+1                       #gentration increment
    return list_ans

              #print accuracy score of that habitate

 this function run BBO for number of generation and population


In [14]:
def main(x_df,y_df):
  res=[]
  for k in range(len(population_size)):
     for m in range(len(generation_size)):
      BBO_ans = runBBO(population_size[k],generation_size[m],x_df,y_df)
      BBO_ans = pd.DataFrame(BBO_ans)
      result=BBO_ans.sort_values(by=1,ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1
      # print(result)
      # result_df.head(10)
      selected_ft=result.iloc[0][0]     #get 0th row 1th habitate(selected_features which gives heigest performans)
      # print("selected features is:",selected_ft)
      # print("Total number of features ",x_df.shape[1])
      # print("total selected Features",len(selected_ft))     #print number of feature
      best_score=result.iloc[0][1]    #calculate best habitate vscore
      # print("accuracy=",best_score)
      res.append([population_size[k],generation_size[m],selected_ft,x_df.shape[1],len(selected_ft),best_score])
  return res;


**Global Search**

In [ ]:
df1=pd.read_csv(r'uploaded_file.csv')

# df1=df1.drop('ID',axis=1)
# print(df1.shape[1])

# from sklearn import preprocessing
# en=preprocessing.LabelEncoder()            #by LabelProcessing
# df1['class']=en.fit_transform(df1['class'])

x1_df = df1.iloc[:, 0:-1]
y1_df= df1.drop(df1.iloc[:, 0:-1], axis=1)
res= main(x1_df,y1_df)
res= pd.DataFrame(res)
res.columns =['Population', 'Generation', 'selected_features','total_features','total_selected_faetues','Accuracy']
# res.to_csv("/content/GlobalResultIteration5.csv")


**Sort Result in Descending Order**


In [ ]:
result=res.sort_values(by='Accuracy',ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1
# print(result)

**Select feature With heigest Acuuracy**

In [ ]:
s1_ft=result.iloc[0][2]     #get 0th row 1th habitate(selected_features which gives heigest performans)
# print("selected features is:",s1_ft)
# print("Total number of features ",x1_df.shape[1])
# print("total selected Features",len(s1_ft))     #print number of feature
best_score=result.iloc[0][5]    #calculate best habitate vscore
# print("accuracy=",best_score)


**This final function is used for Local Search**

In [18]:
def final(best_score,s_ft,max_sc,n1,xxdf,yydf,ft_dataset):
 numft=len(s_ft)
 if(best_score>=max_sc):
  minimize2_df=select_feature_DataFrame(s_ft,xxdf)
  # dataset="With_"+str(numft)+"_Feature_Dataset.csv"   #for create selected feature dataset for each itration
  max_sc=best_score
  ans=main(minimize2_df,yydf)
  ans= pd.DataFrame(ans)
  ans.columns =['Population', 'Generation', 'selected_features','total_features','total_selected_faetues','Accuracy']
  # st="/content/Local_Search_Iteration5_result_"+str(n1)+".csv"     #for create result csv for each itration
  # ans.to_csv(st)
  n1=int(n1)
  n1=n1+1
  result=ans.sort_values(by='Accuracy',ascending=False)  #sort dataframe in descending order based on fitness score by selecting column 1
  select_ft=result.iloc[0][2]     #get 0th row 1th habitate(selected_features which gives heigest performans)
  best_sc=result.iloc[0][5]    #calculate best habitate vscore
  minimize2_df = pd.concat([minimize2_df,y1_df], axis=1)
  # minimize2_df.to_csv(dataset,index=False)        #for generate dataset
  ft_dataset=minimize2_df
  y2_df= minimize2_df.drop(minimize2_df.iloc[:, 0:-1], axis=1)
  final(best_sc,select_ft,max_sc,n1,minimize2_df,yydf,ft_dataset)
 else:
  #dataset="/content/With_"+str(numft)+"_Feature_Dataset_iteration5.csv"   #for create selected feature dataset for each itration
  # output_file = "result.csv"
  # file="final_result.csv"
  ft_dataset.to_csv('result.csv', index=False)        #for generate dataset
  # print("Search is Ended With best Result with "+str(n1)+" Itration")




**Call Final Function for Local Search**

In [ ]:
n=1              #set n=1 to itration How many local search Done
max_score=0       #intialize max_score
final_dataset=pd.DataFrame()
final(best_score,s1_ft,max_score,n,x1_df,y1_df,final_dataset)